**Pengolahan Gambar**

In [ ]:
import cv2

image = cv2.imread('dl.jpg')

gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

inverted_image = cv2.bitwise_not(image)

blurred_image = cv2.GaussianBlur(image, (15, 15), 0)

cv2.imshow('Original Image', image)
cv2.imshow('Grayscale Image', gray_image)
cv2.imshow('Inverted Image', inverted_image)
cv2.imshow('Blurred Image', blurred_image)

cv2.waitKey(0)
cv2.destroyAllWindows()

**Deteksi Gerakan dalam Video**

In [ ]:
import numpy as np
import cv2

cap = cv2.VideoCapture('komputer vision.avi')

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 5.0, (1280, 720))

ret, frame1 = cap.read()
ret, frame2 = cap.read()

print(frame1.shape)

while cap.isOpened():
    diff = cv2.absdiff(frame1, frame2)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(thresh, None, iterations=3)
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if cv2.contourArea(contour) < 900:
            continue
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(frame1, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame1, "Status: {}".format('Movement'), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

    out.write(frame1)

    cv2.imshow("feed", frame1)

    frame1 = frame2
    ret, frame2 = cap.read()

    if not ret:
        break

    if cv2.waitKey(30) & 0xFF == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()

**Pengambilan Gambar Otomatis dengan Deteksi Wajah**

In [ ]:
import os
import cv2
import time
import uuid
import threading

In [ ]:
# isi sesuai dengan lokasi folder di letakkan
IMAGES_PATH = 'D:\Kuliah\Project UAS Komputer Vision\Dokumen\Tensorflow\image'

In [ ]:
# isi sesuai dengan nama gambar yg di input di folder
labels = ['Ayo Berkelahi', 'Linggau Bisa', 'Oke Abangda Mantap', 'Diam']
number_imgs = 15
FRAME_WIDTH = 640
FRAME_HEIGHT = 480
SLEEP_TIME = 1  


face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


def capture_images(label, num_images):
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)
    
    if not cap.isOpened():
        print(f"Failed to open camera for {label}")
        return
    
    print(f'Collecting images for {label}')
    time.sleep(2)  
    
    for imgnum in range(num_images):
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, "Daniel", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        imgname = os.path.join(IMAGES_PATH, label, f'{label}.{str(uuid.uuid4())}.jpg')
        cv2.imwrite(imgname, frame)
        cv2.imshow('frame', frame)
        time.sleep(SLEEP_TIME)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()


for label in labels:
    os.makedirs(os.path.join(IMAGES_PATH, label), exist_ok=True)

for label in labels:
    capture_images(label, number_imgs)